In [3]:
# Remember to pip install pytest

import pandas as pd
from utils import read_cin_data
import pytest

collection_start = pd.to_datetime("01/04/2021", dayfirst=True)
collection_end = pd.to_datetime("31/03/2022", dayfirst=True)

(
    ChildCharacteristics,
    ChildIdentifiers,
    ChildProtectionPlans, # No data in here
    CINdetails,
    CINplanDates, # No data in here
    Header,
    Reviews, # No data in here
    Section47, # No data in here
) = read_cin_data()

We'll try rules as simply doing slices before we try doing them as functions, and before we write tests for them

# Rules to try:
- rule 4180: GenderCurrent (N00097) must be present and valid
- rule 4220: Ethnicity (N00177) must be present and a valid code
- rule 100: ReferenceDate (N00603) must be present and must equal 2022-03-31
- rule 1520: Each pupil UPN (N00001) must be unique across all pupils in the extract
- rule 8510: Each LAchildID (N00097) must be unique across all children within the same LA return. 
- rule 1540: If UPN (N00001) present Characters 5-12 of UPN must be numeric
- rule 8520: PersonBirthDate (N00066) must be on or before ReferenceDate (N00603) or null
- rule 8525Q: Either Date of Birth or Expected Date of Birth must be provided (but not both)
- rule 8600: Child referral date missing or after data collection period

# Harder but better practice (eg needing 2 or more tables):
- rule 8606: CINreferralDate (N00100) cannot be more than 280 days before PersonBirthDate (N00066) or ExpectedPersonBirthDate

# The following are great to do, but can only really be done with appropriate test_xxxx functions as there isn't sufficeint fake data to do them otherwise

In [5]:
# Lets do 4180 as an example.
# For the cin validator, we find failing rows and return the index locations of those rows as a list

valid_gender_codes = ["1", "2", "0", "9"]

df = ChildIdentifiers

condition = df['GenderCurrent'].isin(valid_gender_codes)

failing_rows = list(df[~condition].index)

failing_rows


[9, 21, 32, 45]

In [17]:
# DoB or EDoB, but not both

df = ChildIdentifiers.copy()

# display(ChildIdentifiers)

condition_has_bd = df['PersonBirthDate'].notna()
condition_has_ebd = df['ExpectedPersonBirthDate'].notna()

# ensure only one date is non-null
failing_rows1 = df[~(condition_has_bd ^ condition_has_ebd)].index.tolist()
failing_rows2 = df[condition_has_bd & condition_has_ebd].index.tolist()


print(failing_rows1 + failing_rows2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [16]:
# 4220

valid_ethnicity_codes = ["ABAN","AIND","AOTH","APKN","BAFR","BCRB","BOTH","CHNE","MOTH","MWAS","MWBA","MWBC","NOBT","OOTH","REFU","WBRI","WIRI","WIRT","WOTH","WROM"]

df = ChildCharacteristics

failing_rows = df[~df['Ethnicity'].isin(valid_ethnicity_codes)].index.tolist()
# failing_rows = df[~df['Ethnicity'].astype(str).isin(valid_ethnicity_codes)].index

failing_rows

Index([], dtype='int64')

Lets rewrite these as fucntions so we can pop them inside pytests and so it's more relevant to the validator code. We will do this in a test_XXXX.py file.